In [1]:
print("""
FullName : Diallo Mamadou Korka
Studend’s Number: 022028845F
      
Modelling and Analysis of Complex Networks
Assignment : 9 
Number of the dataset : 3
""")


FullName : Diallo Mamadou Korka
Studend’s Number: 022028845F
      
Modelling and Analysis of Complex Networks
Assignment : 9 
Number of the dataset : 3



In [2]:
# import the modules
import urllib.request
import networkx as nx
import numpy as np
import random
import matplotlib.pyplot as plt

In [3]:
#Load the dataset
# Load the Facebook-Ego network
facebook_url = "https://raw.githubusercontent.com/wang422003/Complex-Networks_exercise/main/Datasets/Group3/Facebook-Ego/348.edges"
urllib.request.urlretrieve(facebook_url, "facebook.edges")
facebook_network = nx.read_edgelist("facebook.edges", nodetype=int)

# Load the Twitter-Ego network
twitter_url = "https://raw.githubusercontent.com/wang422003/Complex-Networks_exercise/main/Datasets/Group3/Twitter-Ego/789071.edges"
urllib.request.urlretrieve(twitter_url, "twitter.edges")
twitter_network = nx.read_edgelist("twitter.edges", nodetype=int, create_using=nx.DiGraph())

In [4]:
# Randomly delete 10% of the edges
import random

# Get the list of edges
edges = list(facebook_network.edges())

# Randomly delete 10% of edges
num_edges_to_remove = int(0.1 * len(edges))
edges_to_remove = random.sample(edges, num_edges_to_remove)

# Remove the edges from the graph
G_removed = facebook_network.copy()
G_removed.remove_edges_from(edges_to_remove)


In [5]:
#Extract Features 
import torch
from torch_geometric.data import Data
from torch_geometric.utils import from_networkx

# Convert the graph to PyTorch Geometric format
data = from_networkx(G_removed)

# Extract additional features as needed
# For example, you can add degree centrality as a feature
data.x = torch.tensor(list(dict(G_removed.degree()).values()), dtype=torch.float).view(-1, 1)


In [13]:
#Train a Machine Learning Model
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, roc_auc_score, precision_score, roc_curve, auc
import matplotlib.pyplot as plt

import torch
from sklearn.model_selection import train_test_split

# Assign binary labels (0 or 1) based on edge presence
if data.y is None:
    data.y = torch.tensor([1 if G_removed.has_edge(e[0], e[1]) else 0 for e in data.edge_index.T], dtype=torch.float)

# Ensure data.y has the correct shape
data.y = data.y.view(-1, 1)

# Convert PyTorch data to NumPy arrays
X = data.x.numpy()
y = data.y.numpy()

# Ensure the shapes are compatible
assert X.shape[0] == y.shape[0], "Number of samples in X and y must be the same"

# Split the data into training and testing sets using indices
indices = np.arange(X.shape[0])
train_indices, test_indices, _, _ = train_test_split(indices, y, test_size=0.2, random_state=42)

# Use indices to get training and testing data
train_data, test_data = X[train_indices], X[test_indices]
y_train, y_test = y[train_indices], y[test_indices]



AssertionError: Number of samples in X and y must be the same

In [ ]:
# Train a machine learning model (Random Forest as an example)
model = RandomForestClassifier()
model.fit(train_data.x.numpy(), y_train)

# Predict on the test set
y_pred = model.predict(test_data.x.numpy())

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
auroc = roc_auc_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)

# Plot ROC curve
fpr, tpr, _ = roc_curve(y_test, y_pred)
roc_auc = auc(fpr, tpr)

In [ ]:
plt.figure()
plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (area = {:.2f})'.format(roc_auc))
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend(loc='lower right')
plt.show()

# Print the results
print("Accuracy:", accuracy)
print("AUROC:", auroc)
print("Precision:", precision)